<a href="https://colab.research.google.com/github/rizabukhari/Agentic-AI-Labs/blob/main/Lab_4_Foundational_Prompt_Engineering_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Lab 4: Foundational Prompt Engineering Patterns**

**Objective:** This lab introduces the fundamental patterns of prompt engineering. You will learn how to shape the model's response by assigning a **persona**, providing examples (**N-shot prompting**), guiding its focus (**directional stimulus**), and enforcing a specific output structure (**template pattern**). These are the building blocks for creating reliable AI solutions in banking.

-----
### Created By: Prashant Sahu
[Connect with me on LinkedIn](https://www.linkedin.com/in/prashantksahu/)

-----
-----
</br>

### **Setup**

First, let's set up our Python environment. This code block will initialize the `gpt-4o-mini` model and a helper function to run our prompts and display the results.

In [ ]:
%%capture
!pip install -q langchain langchain-openai python-dotenv

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
## Load environment variables from local usng .env.
## WARNING: DO NOT RUN THIS CODE IN COLAB !!!
# import os
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
llm = ChatOpenAI(model="gpt-5-nano")

# Helper function to execute prompts and print results
def execute_prompt(prompt_template, title, **kwargs):
    """
    Executes a given prompt template and prints the output neatly.
    """
    print(f"--- {title} ---")

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()

    print("PROMPT:")
    print(prompt.format(**kwargs))

    response = chain.invoke(kwargs)

    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    print(response)
    print("\n" + "-"*50 + "\n")

-----

### **1. The Persona Pattern**

**Background:** The **Persona Pattern** involves instructing the LLM to "act as" a specific role or character. This is one of the most effective ways to control the tone, style, and expertise level of the response. Assigning a persona primes the model to generate content that aligns with the assumed role's knowledge and communication style. 🎭

**Scenario:** A bank needs to draft a marketing email to a high-net-worth individual (HNI) about a new, exclusive investment fund. The tone must be professional, confident, and sophisticated.

In [ ]:
fund_details = """
- Fund Name: "Apex Global Growth Fund"
- Strategy: Invests in high-growth international technology and healthcare equities.
- Target Audience: Sophisticated investors with a high risk appetite.
- Minimum Investment: ₹1 Crore.
"""

#### **Naive Prompt 👎**

Without a persona, the model's tone is generic and lacks the authority required for the target audience.

In [ ]:
naive_persona_prompt = "Write an email about a new investment fund with these details:\n\n{details}"

execute_prompt(
    naive_persona_prompt,
    "1.1 Naive Email Draft",
    details=fund_details
)

--- 1.1 Naive Email Draft ---
PROMPT:
Human: Write an email about a new investment fund with these details:


- Fund Name: "Apex Global Growth Fund"
- Strategy: Invests in high-growth international technology and healthcare equities.
- Target Audience: Sophisticated investors with a high risk appetite.
- Minimum Investment: ₹1 Crore.


==================== RESPONSE ====================

Subject: Introducing Apex Global Growth Fund — A new opportunity in global tech and healthcare equities

Dear Investor,

We are pleased to announce the launch of Apex Global Growth Fund, a new investment vehicle designed for sophisticated investors seeking exposure to high-growth international technology and healthcare equities. The fund employs an active, research-driven approach to identify compelling growth opportunities across developed and emerging markets, with a focus on innovative technology leaders and healthcare pioneers.

Fund highlights
- Fund Name: Apex Global Growth Fund
- Strategy: Invest

*Critique:* This is dry, unprofessional, and fails to convey exclusivity or expertise. It would not appeal to a sophisticated investor.

#### **Persona Pattern Prompt 👍**

By assigning the persona of a **"Senior Wealth Manager,"** we instruct the model to adopt a suitable tone and frame the message appropriately.

In [ ]:
persona_prompt = """
Act as a Senior Wealth Manager at a prestigious private bank.
Your task is to draft a personalized email to a long-standing, high-net-worth client.

The email should be sophisticated, concise, and compelling.
Introduce the new fund and suggest a brief meeting to discuss how it aligns with their portfolio goals.

Use the following fund details:
{details}
"""

execute_prompt(
    persona_prompt,
    "1.2 Persona-Driven Email Draft",
    details=fund_details
)

--- 1.2 Persona-Driven Email Draft ---
PROMPT:
Human: 
Act as a Senior Wealth Manager at a prestigious private bank. 
Your task is to draft a personalized email to a long-standing, high-net-worth client.

The email should be sophisticated, concise, and compelling. 
Introduce the new fund and suggest a brief meeting to discuss how it aligns with their portfolio goals.

Use the following fund details:

- Fund Name: "Apex Global Growth Fund"
- Strategy: Invests in high-growth international technology and healthcare equities.
- Target Audience: Sophisticated investors with a high risk appetite.
- Minimum Investment: ₹1 Crore.



==================== RESPONSE ====================

Subject: Apex Global Growth Fund — A targeted opportunity for your portfolio

Dear [Client Name],

I trust you and your family are well. In appreciation of our long-standing partnership and your growth-focused mandate, I’m pleased to share a new opportunity that aligns with your portfolio objectives.

Introducing 

*Critique:* Excellent. The tone is professional, the language is sophisticated ("investment vehicle," "forward-thinking portfolio strategy"), and the call-to-action is respectful of the client's time.

-----

### **2. The N-Shot Prompting Pattern**

**Background:** This pattern involves providing the model with examples ("shots") to guide its output.

  * **Zero-Shot:** You provide no examples, just the instruction. This works for simple, well-defined tasks.
  * **Few-Shot:** You provide a few examples of input and desired output. This is powerful for teaching the model a specific format, structure, or nuanced classification.

#### **2.1 Zero-Shot Use Case: Customer Request Classification**

**Scenario:** A banking chatbot receives an incoming customer message. It needs to classify the message into a predefined category for routing to the correct department.

In [ ]:
customer_message = "Hi, my credit card was charged twice for a purchase I made at Blinkit yesterday. Can you please help me fix this?"

**Improved Zero-Shot Prompt 👍**
We don't need examples here because the task is a straightforward classification. We just need to give clear instructions and the list of valid categories.

In [ ]:
zeroshot_prompt = """
You are a classification bot. Classify the following customer message into one of these categories:
- Account Inquiry
- Card Dispute
- Loan Application
- Technical Support

Return only the category name.

Message: "{message}"
"""

execute_prompt(
    zeroshot_prompt,
    "2.1 Zero-Shot Classification",
    message=customer_message
)

--- 2.1 Zero-Shot Classification ---
PROMPT:
Human: 
You are a classification bot. Classify the following customer message into one of these categories:
- Account Inquiry
- Card Dispute
- Loan Application
- Technical Support

Return only the category name.

Message: "Hi, my credit card was charged twice for a purchase I made at BigBasket yesterday. Can you please help me fix this?"


==================== RESPONSE ====================

Card Dispute

--------------------------------------------------



*Critique:* This is a perfect use of zero-shot prompting. The task is clear, and the model can easily perform it without needing examples.

#### **2.2 Few-Shot Use Case: Structuring Trade Data**

**Scenario:** A system needs to parse unstructured text from a trader's chat message and convert it into a structured format for the order management system. The format must be exact.

In [ ]:
trader_chat = "we need to buy 5000 shares of RELIANCE at market price asap"

#### **Naive Prompt 👎**

A naive prompt will result in an unpredictable structure. The keys and values might change with every run.

In [ ]:
naive_fewshot_prompt = "Extract the details from this trade message: {message}"

execute_prompt(
    naive_fewshot_prompt,
    "2.2 Naive Trade Extraction",
    message=trader_chat
)

--- 2.2 Naive Trade Extraction ---
PROMPT:
Human: Extract the details from this trade message: we need to buy 5000 shares of RELIANCE at market price asap

==================== RESPONSE ====================

Here are the extracted details from the trade message:

- Action: Buy
- Quantity: 5000 shares
- Stock: RELIANCE
- Price Type: Market price
- Urgency: As soon as possible (ASAP)

--------------------------------------------------



*Critique:* This is just a string, not a machine-readable format. It's also missing a key for the price.

#### **Few-Shot Pattern Prompt 👍**

We provide a clear example to teach the model the **exact output format** we need.

In [ ]:
fewshot_prompt = """
You are an AI that converts unstructured trader chat messages into a structured string. Follow the format of the example precisely.

---
**Example:**
Message: "sell 200 INFY limit 1500"
Output: "ACTION:SELL|TICKER:INFY|QUANTITY:200|PRICE:1500.00"
---

**New Message:**
Message: "{message}"
Output:
"""

execute_prompt(
    fewshot_prompt,
    "2.3 Few-Shot Trade Extraction",
    message=trader_chat
)

--- 2.3 Few-Shot Trade Extraction ---
PROMPT:
Human: 
You are an AI that converts unstructured trader chat messages into a structured string. Follow the format of the example precisely.

---
**Example:**
Message: "sell 200 INFY limit 1500"
Output: "ACTION:SELL|TICKER:INFY|QUANTITY:200|PRICE:1500.00"
---

**New Message:**
Message: "we need to buy 5000 shares of RELIANCE at market price asap"
Output:


==================== RESPONSE ====================

ACTION:BUY|TICKER:RELIANCE|QUANTITY:5000|PRICE:MARKET

--------------------------------------------------



*Critique:* Perfect. The few-shot example forced the model to produce an output that is structured, consistent, and ready for a downstream system to parse.

-----

### **3. The Directional Stimulus Pattern**

**Background:** This pattern guides the LLM to focus on specific aspects of a large piece of text. Instead of a generic summary, you provide "hints" or "directions" to tailor the output to your specific needs. 🎯

**Scenario:** A busy loan officer needs to quickly understand the key risks in a lengthy business credit report. They don't need a full summary, only the negative points.

In [ ]:
credit_report_excerpt = """
Business 'Future Gadgets Ltd.' has shown consistent revenue growth of 15% YoY.
Their cash flow is positive, and they hold significant assets.
However, their debt-to-equity ratio is high at 2.5, and the report notes two recent late payments to major suppliers.
The market outlook for their sector is stable, but dependent on volatile international supply chains.
"""

#### **Naive Prompt 👎**

A generic summary will include positive points, forcing the officer to read through extra information.

In [ ]:
naive_directional_prompt = "Summarize this credit report excerpt:\n\n{report}"

execute_prompt(
    naive_directional_prompt,
    "3.1 Naive Report Summary",
    report=credit_report_excerpt
)

--- 3.1 Naive Report Summary ---
PROMPT:
Human: Summarize this credit report excerpt:


Business 'Future Gadgets Ltd.' has shown consistent revenue growth of 15% YoY. Their cash flow is positive, and they hold significant assets. However, their debt-to-equity ratio is high at 2.5, and the report notes two recent late payments to major suppliers. The market outlook for their sector is stable, but dependent on volatile international supply chains.


==================== RESPONSE ====================

Future Gadgets Ltd. has experienced steady revenue growth of 15% year-over-year and maintains positive cash flow, along with substantial assets. However, the company has a high debt-to-equity ratio of 2.5 and has made two recent late payments to major suppliers. The market outlook for their industry is stable, although it relies on fluctuating international supply chains.

--------------------------------------------------



*Critique:* The summary is accurate but mixes positive and negative points.

#### **Directional Stimulus Prompt 👍**

We give the model a clear "hint" to focus *only* on the risk factors.

In [ ]:
directional_prompt = """
Summarize the following credit report excerpt in 2-3 bullet points.

Hint: Focus exclusively on the negative points and potential risk factors mentioned in the report.

Report:
{report}
"""

execute_prompt(
    directional_prompt,
    "3.2 Directional Risk Summary",
    report=credit_report_excerpt
)

--- 3.2 Directional Risk Summary ---
PROMPT:
Human: 
Summarize the following credit report excerpt in 2-3 bullet points.

Hint: Focus exclusively on the negative points and potential risk factors mentioned in the report.

Report:

Business 'Future Gadgets Ltd.' has shown consistent revenue growth of 15% YoY. Their cash flow is positive, and they hold significant assets. However, their debt-to-equity ratio is high at 2.5, and the report notes two recent late payments to major suppliers. The market outlook for their sector is stable, but dependent on volatile international supply chains.



==================== RESPONSE ====================

- Future Gadgets Ltd. has a high debt-to-equity ratio of 2.5, indicating a reliance on debt which poses a financial risk.
- The company has recorded two recent late payments to major suppliers, suggesting potential cash flow management issues.
- Their market outlook is stable but heavily dependent on volatile international supply chains, introducing 

*Critique:* This is much more effective. It provides the loan officer with a concise, focused list of the exact information they need, saving them time and improving efficiency.

-----

### **4. The Template & Meta Language Pattern**

**Background:** This powerful pattern involves defining a strict structure or a custom "language" for the model's output. By providing a template, you force the LLM to populate its response within your predefined format, making the output predictable and machine-readable. This is essential for JSON, XML, or other structured data needs. 📝

**Scenario:** A bank needs to generate a standardized risk assessment for a credit card application in a JSON format that can be logged directly into their system.

In [ ]:
applicant_data = """
- Name: Rahul Verma
- Age: 28
- Annual Income: ₹8,00,000
- CIBIL Score: 760
- Existing Debt: ₹1,50,000 (Personal Loan)
"""

#### **Naive Prompt 👎**

Without a strict template, the model will generate JSON, but the structure, keys, and value types will be inconsistent.

In [ ]:
naive_template_prompt = "Create a JSON risk summary for this credit card applicant:\n\n{data}"

execute_prompt(
    naive_template_prompt,
    "4.1 Naive JSON Generation",
    data=applicant_data
)

--- 4.1 Naive JSON Generation ---
PROMPT:
Human: Create a JSON risk summary for this credit card applicant:


- Name: Rahul Verma
- Age: 28
- Annual Income: ₹8,00,000
- CIBIL Score: 760
- Existing Debt: ₹1,50,000 (Personal Loan)


==================== RESPONSE ====================

Here’s a JSON risk summary for the credit card applicant Rahul Verma based on the provided information:

```json
{
  "applicant": {
    "name": "Rahul Verma",
    "age": 28,
    "annual_income": 800000,
    "cibil_score": 760,
    "existing_debt": 150000,
    "debt_to_income_ratio": 0.1875,
    "risk_assessment": {
      "cibil_score_risk": "Low",
      "existing_debt_risk": "Moderate",
      "overall_risk": "Low",
      "recommendation": "Eligible for credit card with standard terms."
    }
  }
}
```

### Explanation:
- **Debt-to-Income Ratio (DTI)**: Calculated as existing debt divided by annual income (₹1,50,000 / ₹8,00,000 = 0.1875 or 18.75%). This helps assess the burden of existing debt.
- **Risk Asses

*Critique:* The keys (`applicant`, `income`, `risk`) are arbitrary, and the `risk` value is a subjective string, not a standardized category.

#### **Template Pattern Prompt 👍**

We define a "meta language" by specifying the exact keys and the allowed values for certain fields, ensuring a perfectly structured output.

In [ ]:
template_prompt = """
Act as a risk assessment bot. Generate a JSON summary for the following credit card applicant.

The output must be a valid JSON object that strictly follows this template:
- "applicantId": string (use applicant's name)
- "cibilScore": integer
- "incomeToDebtRatio": float (calculate as Annual Income / Existing Debt)
- "riskLevel": string (must be one of the following: "Low", "Medium", "High")
- "isApproved": boolean (approve if score > 750 and incomeToDebtRatio > 5)

Applicant Data:
{data}
"""

execute_prompt(
    template_prompt,
    "4.2 Template-Driven JSON Generation",
    data=applicant_data
)

--- 4.2 Template-Driven JSON Generation ---
PROMPT:
Human: 
Act as a risk assessment bot. Generate a JSON summary for the following credit card applicant.

The output must be a valid JSON object that strictly follows this template:
- "applicantId": string (use applicant's name)
- "cibilScore": integer
- "incomeToDebtRatio": float (calculate as Annual Income / Existing Debt)
- "riskLevel": string (must be one of the following: "Low", "Medium", "High")
- "isApproved": boolean (approve if score > 750 and incomeToDebtRatio > 5)

Applicant Data:

- Name: Rahul Verma
- Age: 28
- Annual Income: ₹8,00,000
- CIBIL Score: 760
- Existing Debt: ₹1,50,000 (Personal Loan)



==================== RESPONSE ====================

```json
{
  "applicantId": "Rahul Verma",
  "cibilScore": 760,
  "incomeToDebtRatio": 5.33,
  "riskLevel": "Medium",
  "isApproved": true
}
```

--------------------------------------------------



*Critique:* This is perfect. The JSON is clean, follows the exact schema, performs a calculation as instructed, and uses a standardized value for `riskLevel`. This output can be reliably processed by another program.